In [12]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim


In [8]:
# ---------- Data ----------
def generate_time_series_data(samples, timesteps, features):
    X = np.random.rand(samples, timesteps, features).astype(np.float32)
    y = np.sum(X, axis=2)[:, -1].reshape(-1, 1)  # target = sum of last timestep
    return torch.from_numpy(X), torch.from_numpy(y)

X, y = generate_time_series_data(samples=1000, timesteps=10, features=2)

In [9]:
X

tensor([[[0.8669, 0.7431],
         [0.1735, 0.6549],
         [0.6133, 0.0882],
         ...,
         [0.2493, 0.8621],
         [0.5048, 0.7832],
         [0.1657, 0.5222]],

        [[0.6127, 0.6289],
         [0.8480, 0.1342],
         [0.6371, 0.0681],
         ...,
         [0.7495, 0.9593],
         [0.1693, 0.4477],
         [0.4741, 0.2324]],

        [[0.8222, 0.1830],
         [0.2762, 0.5665],
         [0.8521, 0.1222],
         ...,
         [0.7275, 0.9126],
         [0.7390, 0.2893],
         [0.3775, 0.8459]],

        ...,

        [[0.8739, 0.6744],
         [0.2480, 0.8189],
         [0.4200, 0.6663],
         ...,
         [0.7915, 0.7773],
         [0.5981, 0.0674],
         [0.5378, 0.2118]],

        [[0.4561, 0.8353],
         [0.4427, 0.7039],
         [0.9481, 0.1668],
         ...,
         [0.6151, 0.9097],
         [0.8367, 0.8171],
         [0.1066, 0.0636]],

        [[0.1010, 0.0231],
         [0.2905, 0.5338],
         [0.0864, 0.3666],
         ...,
 

In [ ]:
# ---------- Model ----------
class ConvLSTM1DCell(nn.Module):
    def __init__(self, input_dim, filters, kernel_size, padding='same'):
        super(ConvLSTM1DCell, self).__init__()
        self.input_dim = input_dim
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2 if padding == 'same' else 0

        self.conv_f = nn.Conv1d(input_dim + filters, filters, kernel_size, padding=self.padding)
        self.conv_i = nn.Conv1d(input_dim + filters, filters, kernel_size, padding=self.padding)
        self.conv_c = nn.Conv1d(input_dim + filters, filters, kernel_size, padding=self.padding)
        self.conv_o = nn.Conv1d(input_dim + filters, filters, kernel_size, padding=self.padding)

        self.peephole_f = nn.Conv1d(filters, filters, kernel_size=1)
        self.peephole_i = nn.Conv1d(filters, filters, kernel_size=1)
        self.peephole_o = nn.Conv1d(filters, filters, kernel_size=1)

    def forward(self, x_t, states):
        h_prev, c_prev = states
        combined = torch.cat([x_t, h_prev], dim=1)
        f_t = torch.sigmoid(self.conv_f(combined) + self.peephole_f(c_prev))
        i_t = torch.sigmoid(self.conv_i(combined) + self.peephole_i(c_prev))
        c_tilde = torch.tanh(self.conv_c(combined))
        c_t = f_t * c_prev + i_t * c_tilde
        o_t = torch.sigmoid(self.conv_o(combined) + self.peephole_o(c_t))
        h_t = o_t * torch.tanh(c_t)
        return h_t, (h_t, c_t)

class ConvLSTM1D(nn.Module):
    def __init__(self, input_dim, filters, kernel_size, return_sequences=False, padding='same'):
        super(ConvLSTM1D, self).__init__()
        self.cell = ConvLSTM1DCell(input_dim, filters, kernel_size, padding)
        self.return_sequences = return_sequences

    def forward(self, x, hidden_state=None):
        x = x.permute(0, 2, 1)  # (batch, features, seq_len)
        batch_size, input_dim, seq_len = x.size()

        h_t = torch.zeros(batch_size, self.cell.filters, 1, device=x.device)
        c_t = torch.zeros(batch_size, self.cell.filters, 1, device=x.device)

        outputs = []
        for t in range(seq_len):
            x_t = x[:, :, t:t+1]  # (batch, features, 1)
            h_t, (h_t, c_t) = self.cell(x_t, (h_t, c_t))
            outputs.append(h_t)

        outputs = torch.stack(outputs, dim=1)  # (batch, seq_len, filters, seq_len)
        if not self.return_sequences:
            outputs = outputs[:, -1, :, :]  # (batch, filters, seq_len)
        return outputs, (h_t, c_t)


In [13]:
# ---------- Training ----------
class ConvLSTMModel(nn.Module):
    def __init__(self, input_dim, filters, kernel_size):
        super().__init__()
        self.convlstm = ConvLSTM1D(input_dim, filters, kernel_size)
        self.fc = nn.Linear(filters, 1)  # Final regression layer

    def forward(self, x):
        out, _ = self.convlstm(x)
        out = out.mean(dim=2)  # Global average pooling over sequence length
        return self.fc(out)

model = ConvLSTMModel(input_dim=8, filters=16, kernel_size=3)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
X, y = X.to(device), y.to(device)
print(device)

cuda


In [16]:
# ---------- Training Loop ----------
for epoch in range(1, 21):
    model.train()
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch:02d} - Loss: {loss.item():.4f}")

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 10 for tensor number 1 in the list.